# Speedtest data exploration

### How data is  collected

Running speedtest application (?) similar to speedtest.net Calculating upload, download speed + png latency

### How data looks like
Load libraries:

In [1]:
from data_exploration import *

In [2]:
import pyodbc
def connect_to_mssql():
    with open('../credentials.json', 'r') as f_credentials:
        credentials_config = json.load(f_credentials)
    password=credentials_config['mssql_password']
    srv=credentials_config['mssql_host']
    connection = pyodbc.connect(driver='/usr/local/lib/libtdsodbc.so', server=srv,port='1433', database='net_speed_md', uid='cybera_sql', pwd=password)
    return connection

Set up test time interval:

In [3]:
time_interval='4w' #5d

In [4]:
#Set up starting point, by default if will start from current time
starting_point=datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#starting point="2019-01-10 14:00:00"  # to set upl alternative starting point
print("Starting point:",starting_point )

Starting point: 2019-02-04 16:56:47


Set up influxdb connection:

In [5]:
client, client_df = connect_to_influxdb()

Checking last 10 records for the SPEEDTEST_UPLOAD  measurment and see how data looks like:

In [6]:
query_upload = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' ORDER BY time DESC LIMIT 10;"
upload_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload,table_name='SPEEDTEST_UPLOAD')
upload_df

,time,IP,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER,UPLOAD
7,2019-01-23 11:12:12-06:00,199.116.232.249,02-99-03-82-52-06,Cybera,AB),3,Axia Connect Ltd (Calgary,125.48
2,2019-01-23 11:12:05-06:00,206.45.204.107,02-0e-04-02-e2-e0,Bell MTS,MB),5,Rogers (Winnipeg,0.62
5,2019-01-23 11:12:08-06:00,204.112.96.66,02-07-06-80-d4-7c,Bell MTS,MB),7,Voyageur Internet (Winnipeg,4.36
3,2019-01-23 11:12:07-06:00,204.9.14.110,02-0e-04-82-02-40,Xplornet Communications,MB),9,RFNow Inc. (Virden,41.23
6,2019-01-23 11:12:09-06:00,199.19.63.140,02-84-05-80-b0-f2,High Speed Crow,MB),12,LES.NET (Winnipeg,5.63
9,2019-01-23 11:12:14-06:00,162.249.42.115,02-08-02-c1-c1-34,Manitoba Hydro International,MB),14,Morenet (Morden,115.91
1,2019-01-23 11:12:04-06:00,204.112.104.114,02-cd-07-40-b1-3f,Bell MTS,MB),15,Commstream Communications (Winnipeg,40.71
8,2019-01-23 11:12:13-06:00,216.130.92.150,02-43-08-c0-b1-81,Bell MTS,MB),16,Valley Fiber (Winkler,3.91
0,2019-01-23 11:12:04-06:00,206.188.114.98,02-06-09-01-32-ac,Keewaytinook Okimakanak,MB),17,Westman Communications (Brandon,47.58
4,2019-01-23 11:12:08-06:00,206.188.114.178,02-8d-07-c1-a3-26,Keewaytinook Okimakanak,MB),18,Voyageur Internet (Winnipeg,9.57


Let's take just one device, for example 3:

In [7]:
query_upload = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND SK_PI='3' ORDER BY time DESC LIMIT 10;"
upload_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload,table_name='SPEEDTEST_UPLOAD')
upload_df

,time,IP,PI_MAC,PROVIDER,PROVINCE,SK_PI,TEST_SERVER,UPLOAD
0,2019-01-22 01:54:08-06:00,199.116.232.249,02-99-03-82-52-06,Cybera,AB),3,TELUS Mobility (Calgary,127.83
1,2019-01-22 05:36:05-06:00,199.116.232.249,02-99-03-82-52-06,Cybera,AB),3,Axia Connect Ltd (Calgary,123.99
2,2019-01-22 09:18:07-06:00,199.116.232.249,02-99-03-82-52-06,Cybera,AB),3,Axia Connect Ltd (Calgary,124.01
3,2019-01-22 13:00:10-06:00,199.116.232.249,02-99-03-82-52-06,Cybera,AB),3,Axia Connect Ltd (Calgary,123.13
4,2019-01-22 16:42:05-06:00,199.116.232.249,02-99-03-82-52-06,Cybera,AB),3,Axia Connect Ltd (Calgary,121.87
5,2019-01-22 20:24:17-06:00,199.116.232.249,02-99-03-82-52-06,Cybera,AB),3,Axia Connect Ltd (Calgary,126.85
6,2019-01-23 00:06:12-06:00,199.116.232.249,02-99-03-82-52-06,Cybera,AB),3,Axia Connect Ltd (Calgary,124.77
7,2019-01-23 03:48:19-06:00,199.116.232.249,02-99-03-82-52-06,Cybera,AB),3,Axia Connect Ltd (Calgary,127.03
8,2019-01-23 07:30:02-06:00,199.116.232.249,02-99-03-82-52-06,Cybera,AB),3,Axia Connect Ltd (Calgary,123.48
9,2019-01-23 11:12:12-06:00,199.116.232.249,02-99-03-82-52-06,Cybera,AB),3,Axia Connect Ltd (Calgary,125.48


Let's compare with what we have in MS SQL database:

In [8]:
cnxn = connect_to_mssql()
sql = "SELECT TOP 10 * FROM FCT_SPEEDTEST WHERE PROVIDER!='iperf' AND SK_PI='3' ORDER BY DATA_DATE DESC;"
pd.read_sql(sql,cnxn)

,SK_PI,SK_FILE_PATTERN,DATA_DATE,PROVIDER,IP,TEST_SERVER,PROVINCE,PING,DOWNLOAD,UPLOAD
0,3,19,2019-01-23 17:12:12.533,Cybera,199.116.232.249,Axia Connect Ltd (Calgary,AB),10.576,179.46,125.48
1,3,19,2019-01-23 13:30:02.203,Cybera,199.116.232.249,Axia Connect Ltd (Calgary,AB),13.392,180.01,123.48
2,3,19,2019-01-23 09:48:19.900,Cybera,199.116.232.249,Axia Connect Ltd (Calgary,AB),10.480,178.92,127.03
3,3,19,2019-01-23 06:06:12.573,Cybera,199.116.232.249,Axia Connect Ltd (Calgary,AB),10.700,179.67,124.77
4,3,19,2019-01-23 02:24:17.257,Cybera,199.116.232.249,Axia Connect Ltd (Calgary,AB),10.528,183.38,126.85
5,3,19,2019-01-22 22:42:05.927,Cybera,199.116.232.249,Axia Connect Ltd (Calgary,AB),16.289,182.74,121.87
6,3,19,2019-01-22 19:00:10.610,Cybera,199.116.232.249,Axia Connect Ltd (Calgary,AB),10.562,179.49,123.13
7,3,19,2019-01-22 15:18:07.287,Cybera,199.116.232.249,Axia Connect Ltd (Calgary,AB),10.527,180.59,124.01
8,3,19,2019-01-22 11:36:05.967,Cybera,199.116.232.249,Axia Connect Ltd (Calgary,AB),10.687,180.22,123.99
9,3,19,2019-01-22 07:54:08.647,Cybera,199.116.232.249,TELUS Mobility (Calgary,AB),49.079,142.51,127.83


Are there any zeros in mssql database?

In [9]:
sql = "SELECT * FROM FCT_SPEEDTEST WHERE PROVIDER!='iperf' AND (UPLOAD=0 OR DOWNLOAD=0 OR PING=0);"
pd.read_sql(sql,cnxn)

,SK_PI,SK_FILE_PATTERN,DATA_DATE,PROVIDER,IP,TEST_SERVER,PROVINCE,PING,DOWNLOAD,UPLOAD
0,7,19,2018-12-13 20:42:03.723,Bell MTS,204.112.96.66,Commstream Communications (Winnipeg,MB),1800000.000,0.00,0.00
1,4,19,2018-11-24 21:36:08.367,Bell MTS,204.112.96.94,Voyageur Internet (Winnipeg,MB),67.381,1.19,0.00
2,4,19,2018-11-27 05:06:15.177,Bell MTS,204.112.96.94,Shaw Communications (Winnipeg,MB),68.232,0.00,0.65
3,4,19,2018-11-30 14:30:05.207,Bell MTS,204.112.96.94,Shaw Communications (Winnipeg,MB),78.877,1.01,0.00


Something is possibly wrong with these measurments, we wille exclude zeros for now.

### How may data points per device?

Getting device numbers(tags SK_PI):

In [10]:
device_numbers=get_tag_values_influxdb(client_influx=client,table_name='SPEEDTEST_UPLOAD', tag_name='SK_PI')
device_numbers=list(map(int, device_numbers))
device_numbers= sorted(device_numbers)
print(device_numbers)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18]


Getting number of data points per device for the entire period of time.

In [11]:
query_upload_counts = "SELECT COUNT(UPLOAD) FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND time<= '"+starting_point+"' AND UPLOAD>0 GROUP BY SK_PI;"
upload_counts=get_stats_influxdb(client_influx=client,
                               query_influx=query_upload_counts,
                               stat_name='count',
                               device_numbers=device_numbers)

Plotting device numbers and number of data points. 

In [12]:
simple_bar_plot(xvalues=device_numbers,
                yvalues=upload_counts,
                name="upload datapoints",
                title="Number of data points per device to the date "+starting_point,
                ytitle="Number of datapoints")

Some of the devices have small number of datapoints, may be they are just installed? Lets check how many dataponts came in last 4 weeks.

Getting number of datapoints per device in last 4 weeks.

In [13]:
query_upload_counts_time = "SELECT COUNT(UPLOAD) FROM SPEEDTEST_UPLOAD WHERE time >= '"+starting_point+"'-"+time_interval+" AND PROVIDER!='iperf' AND UPLOAD>0 GROUP BY SK_PI ;"
upload_counts_time = get_stats_influxdb(client_influx=client,
                                      query_influx=query_upload_counts_time,
                                      stat_name='count',
                                      device_numbers=device_numbers)

Plotting combined barchart - entire number of datapoints vs number of datapoints in last 4 weeks.

In [14]:
combined_bar_plot_2traces(xvalues=device_numbers,
                          yvalues1=upload_counts_time,
                          yvalues2=[a - b for a, b in zip(upload_counts, upload_counts_time)],
                          name1='Last '+time_interval,
                          name2='The rest of the time',
                          title="Comparing number of datapoints in last "+time_interval+" vs entire time starting from "+starting_point,
                          ytitle="Number of datapoints")

Devices 1,2,4,6 and 8 have data but not in the last 4 weeks.Let's check last reporting time for every device.

In [15]:
query_upload_last = "SELECT LAST(UPLOAD), time FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND time <= '"+starting_point+"' AND UPLOAD>0 GROUP BY SK_PI;"
result_upload_last=get_stats_influxdb(client_influx=client,
                               query_influx=query_upload_last,
                               stat_name='time',
                               device_numbers=device_numbers)

In [16]:
query_upload_first = "SELECT FIRST(UPLOAD), time FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf' AND time <= '"+starting_point+"' AND UPLOAD>0 GROUP BY SK_PI;"
result_upload_first=get_stats_influxdb(client_influx=client,
                               query_influx=query_upload_first,
                               stat_name='time',
                               device_numbers=device_numbers)

In [17]:
print("Speedtest reporting times:")
data=[]
for i in range(len(device_numbers)):
    try:
        result_upload_first[i] = dateutil.parser.parse(result_upload_first[i]).strftime('%Y-%m-%d %H:%M:%S')
    except:
        result_upload_first[i]=None
    try:    
        result_upload_last[i] = dateutil.parser.parse(result_upload_last[i]).strftime('%Y-%m-%d %H:%M:%S')
    except:
        result_upload_last[i]=None
    print("Device: ", device_numbers[i],"  was reporting from ", result_upload_first[i], " to ",result_upload_last[i])
    trace = go.Scatter(x=[result_upload_first[i],result_upload_last[i]],y=[device_numbers[i],device_numbers[i]], 
                       name = device_numbers[i],marker=dict(color=colors[i]))
    data.append(trace)
layout = dict(title = "Device reporting times(speedtest)",xaxis=dict(title="Time"),
        yaxis=dict(title="Device Number"))
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Speedtest reporting times:
Device:  1   was reporting from  2018-10-10 15:45:43  to  2018-10-11 12:00:09
Device:  2   was reporting from  2018-10-11 15:55:58  to  2018-11-03 15:00:12
Device:  3   was reporting from  2018-10-15 19:36:18  to  2019-01-23 17:12:12
Device:  4   was reporting from  2018-10-27 00:03:48  to  2018-12-05 16:36:12
Device:  5   was reporting from  2018-10-26 18:18:08  to  2019-01-23 17:12:05
Device:  6   was reporting from  2018-10-27 20:12:15  to  2018-10-31 13:00:10
Device:  7   was reporting from  2018-11-11 00:36:13  to  2019-01-23 17:12:08
Device:  8   was reporting from  2018-11-07 18:22:07  to  2019-01-14 11:12:13
Device:  9   was reporting from  2018-11-30 18:12:12  to  2019-01-23 17:12:07
Device:  10   was reporting from  2018-12-12 22:29:40  to  2019-01-23 06:06:09
Device:  11   was reporting from  2018-11-30 18:12:09  to  2019-01-23 17:12:02
Device:  12   was reporting from  2018-11-30 18:12:06  to  2019-01-23 17:12:09
Device:  14   was reporting from  

### Statistics by device

In [18]:
query_download = "SELECT * FROM SPEEDTEST_DOWNLOAD WHERE PROVIDER!='iperf'AND DOWNLOAD>0 AND time >='"+starting_point+"'-"+\
                         time_interval;
download_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_download,
                                          table_name='SPEEDTEST_DOWNLOAD')

In [19]:
query_upload = "SELECT * FROM SPEEDTEST_UPLOAD WHERE PROVIDER!='iperf'AND UPLOAD>0 AND time >= '"+starting_point+"'-"+\
                         time_interval;
upload_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_upload,
                                          table_name='SPEEDTEST_UPLOAD')

In [20]:
query_ping = "SELECT * FROM SPEEDTEST_PING WHERE PROVIDER!='iperf'AND PING>0 AND time >= '"+starting_point+"'-"+\
                         time_interval;
ping_df = get_dataframe_from_influxdb(client_df=client_df,query_influx=query_ping,
                                          table_name='SPEEDTEST_PING')
ping_df = ping_df[ping_df.PING != 1800000.000] ## Removed outier that was found in the ms sql table

In [21]:
download_summary=mean_max_median_by1(download_df,'DOWNLOAD')
device_numbers_d=download_summary["SK_PI"].unique()
download_line=go.Scatter(x=device_numbers_d,y=[50] * len(device_numbers_d), mode='markers',marker=dict(color='red'), name='50Mps')
combined_bar_plot_3traces(xvalues=download_summary["SK_PI"],
                         yvalues1=download_summary["max"],
                         yvalues2=download_summary["mean"],
                         yvalues3=download_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Download speed over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Mbps",
                         line=download_line,
                         stack=False)

In [22]:
upload_summary=mean_max_median_by1(upload_df,'UPLOAD')
device_numbers_u=upload_summary["SK_PI"].unique()
upload_line=go.Scatter(x=device_numbers_u,y=[10] * len(device_numbers_u), mode='markers',marker=dict(color='red'), name='10Mbps')

combined_bar_plot_3traces(xvalues=upload_summary["SK_PI"],
                         yvalues1=upload_summary["max"],
                         yvalues2=upload_summary["mean"],
                         yvalues3=upload_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Upload speed over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Mbps",
                         line=upload_line,
                         stack=False)

In [23]:
ping_summary=mean_max_median_by1(ping_df,'PING')
combined_bar_plot_3traces(xvalues=ping_summary["SK_PI"],
                         yvalues1=ping_summary["max"],
                         yvalues2=ping_summary["mean"],
                         yvalues3=ping_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Ping latency over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Miliseconds",
                         stack=False)

In [24]:
simple_boxplot(dataframe=download_df,plot_value='DOWNLOAD',sort_value='SK_PI',
               title="Download speed over the last "+time_interval+ " starting from "+ starting_point, 
               ytitle="Mbps")

In [25]:
simple_boxplot(dataframe=upload_df,plot_value='UPLOAD',sort_value='SK_PI',
               title="Upload speed over the last "+time_interval+ " starting from "+ starting_point, 
               ytitle="Mbps")

In [26]:
simple_boxplot(dataframe=ping_df,plot_value='PING',sort_value='SK_PI',
               title="Ping latency over the last "+time_interval+ " starting from "+ starting_point, 
               ytitle="Miliseconds")

### Test servers
Are there any NaNs or empty strings?

In [27]:
print(sum(pd.isnull(upload_df["TEST_SERVER"])))
print(sum(pd.isnull(upload_df["PROVINCE"])))
print(sum(upload_df["TEST_SERVER"] == ''))
print(sum(upload_df["PROVINCE"] == ''))

0
0
0
0


In [28]:
print("There are ",len(upload_df["PROVINCE"].unique()), " provinces: ", upload_df["PROVINCE"].unique(), "\n")
print("There are ",len(upload_df["TEST_SERVER"].unique()), " test servers", upload_df["TEST_SERVER"].unique(), "\n")

pd.options.mode.chained_assignment = None
upload_df["server"] = upload_df["TEST_SERVER"] + " " +upload_df["PROVINCE"]
print("There are ",len(upload_df["server"].unique()), " test servers+provinces", upload_df["server"].unique(), "\n")

There are  4  provinces:  [' AB)' ' MB)' ' SK)' ' ND)'] 

There are  21  test servers ['Cybera (Calgary' 'Axia Connect Ltd (Calgary' 'TELUS Mobility (Calgary'
 'Voyageur Internet (Winnipeg' 'Rogers (Winnipeg'
 'Shaw Communications (Winnipeg' 'Commstream Communications (Winnipeg'
 'LES.NET (Winnipeg' 'Fastnet Communications (Winnipeg'
 'Valley Fiber (Winkler' 'Morenet (Morden'
 'VOI Network Solutions (Winnipeg' 'RFNow Inc. (Virden'
 'BlackSun Inc. (Saskatoon' 'Westman Communications (Brandon'
 'SaskTel (Regina' 'Access Communications Co-operative Limited (Regina'
 'Bell Mobility (Winnipeg' 'Midco (Minot' 'NetSet Communications (Brandon'
 'RFNow Inc (Brandon'] 

There are  21  test servers+provinces ['Cybera (Calgary  AB)' 'Axia Connect Ltd (Calgary  AB)'
 'TELUS Mobility (Calgary  AB)' 'Voyageur Internet (Winnipeg  MB)'
 'Rogers (Winnipeg  MB)' 'Shaw Communications (Winnipeg  MB)'
 'Commstream Communications (Winnipeg  MB)' 'LES.NET (Winnipeg  MB)'
 'Fastnet Communications (Winnipeg  MB

#### How many test servers per device?

In [29]:
pd.options.mode.chained_assignment = None
upload_df["server"] = upload_df["TEST_SERVER"] + " " +upload_df["PROVINCE"]
download_df["server"] = download_df["TEST_SERVER"] + " " +download_df["PROVINCE"]
ping_df["server"] = ping_df["TEST_SERVER"] + " " +ping_df["PROVINCE"]

points_by_device=upload_df.groupby(['SK_PI']).size().reset_index(name='counts').sort_values('SK_PI')

test_servers_per_device = upload_df.groupby('SK_PI').apply(lambda x: x['server'].unique()).apply(pd.Series)
test_servers_per_device['server_count'] = test_servers_per_device.apply(lambda x: x.count(), axis=1)
test_server_counts_per_device = pd.Series(test_servers_per_device['server_count']).value_counts().reset_index()
points_by_device_by_server=upload_df.groupby(['SK_PI', 'server']).size().reset_index()

In [30]:
simple_bar_plot(xvalues=test_servers_per_device.index,
                yvalues=test_servers_per_device["server_count"],
                name = "Number of providers",title="Number of test servers per device over the last "+time_interval+ " starting from "+ starting_point,
                ytitle="Number of providers")

In [31]:
simple_pie_chart(labels=test_server_counts_per_device['index'],
            values=test_server_counts_per_device['server_count'],
                 title="Number of test servers per device over the last "+time_interval+ " starting from "+ starting_point,)

In [32]:
combined_bar_plot_multitraces(dataframe=points_by_device_by_server,
                              device_numbers=device_numbers,
                              points_by_device=points_by_device,
                              sort_value='server',
                              ytitle="Percentage of data points",
                              title="Percentage of data points per device per test server over the last "+time_interval+ " starting from "+ starting_point)

In [33]:
upload_summary=mean_max_median_by1(upload_df,'UPLOAD',index_col='server')
upload_line=go.Scatter(x=upload_summary["server"].unique(),y=[10] * len(upload_summary["server"].unique()), mode='markers',marker=dict(color='red'), name='10Mps')
combined_bar_plot_3traces(xvalues=upload_summary["server"],
                         yvalues1=upload_summary["max"],
                         yvalues2=upload_summary["mean"],
                         yvalues3=upload_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Upload speed by test server over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Mbps",
                         xtitle="Provider",
                         line=upload_line,
                         stack=False,
                         margin=True)

In [34]:
download_summary=mean_max_median_by1(download_df,'DOWNLOAD',index_col='server')
download_line=go.Scatter(x=upload_summary['server'].unique(),y=[50] * len(upload_summary['server'].unique()), mode='markers',marker=dict(color='red'), name='50Mps')
combined_bar_plot_3traces(xvalues=download_summary['server'],
                         yvalues1=download_summary["max"],
                         yvalues2=download_summary["mean"],
                         yvalues3=download_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Download speed by test server over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Mbps",
                         xtitle="Test server",
                         line=download_line,
                         stack=False,
                         margin=True)

In [35]:
ping_summary=mean_max_median_by1(ping_df,'PING',index_col='server')
combined_bar_plot_3traces(xvalues=ping_summary['server'],
                         yvalues1=ping_summary["max"],
                         yvalues2=ping_summary["mean"],
                         yvalues3=ping_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Ping latency by  test server over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Miliseconds",
                         xtitle="Test server",
                         stack=False,
                         margin=True)

### Providers
Are there any NaNs or empty strings?

In [36]:
print(sum(pd.isnull(upload_df["PROVIDER"])))
print(sum(upload_df["PROVIDER"] == ''))

0
0


In [37]:
print("There are ",len(upload_df["PROVIDER"].unique()), " providers", upload_df["PROVIDER"].unique(), "\n")

There are  10  providers ['Cybera' 'Bell MTS' 'Commstream Communications' 'MERLIN'
 'Xplornet Communications' 'TeraGo Networks' 'Cogent Communications'
 'High Speed Crow' 'Manitoba Hydro International'
 'Keewaytinook Okimakanak'] 



#### How many providers per device?

In [38]:
points_by_device=upload_df.groupby(['SK_PI']).size().reset_index(name='counts').sort_values('SK_PI')
providers_per_device = upload_df.groupby('SK_PI').apply(lambda x: x["PROVIDER"].unique()).apply(pd.Series)
providers_per_device['provider_count'] = providers_per_device.apply(lambda x: x.count(), axis=1)
provider_counts_per_device = pd.Series(providers_per_device['provider_count']).value_counts().reset_index()
points_by_device_by_provider=upload_df.groupby(['SK_PI', 'PROVIDER']).size().reset_index()

In [39]:
simple_bar_plot(xvalues=providers_per_device.index,
                yvalues=providers_per_device["provider_count"],
                name = "Number of providers",title="Number of providers per device over the last "+time_interval + " starting from "+ starting_point,
                ytitle="Number of providers")

In [40]:
simple_pie_chart(labels=provider_counts_per_device['index'],
            values=provider_counts_per_device["provider_count"],title="Number of providers per device over the last "+time_interval+ " starting from "+ starting_point)

In [41]:
combined_bar_plot_multitraces(dataframe=points_by_device_by_provider,
                              device_numbers=device_numbers,
                              points_by_device=points_by_device,
                              sort_value='PROVIDER',
                              ytitle="Percentage of data",
                              title="Percentage of data points per device per service provider over the last "+time_interval+ " starting from "+ starting_point,)

In [50]:
upload_summary=mean_max_median_by1(upload_df,'UPLOAD',index_col='PROVIDER')
upload_line=go.Scatter(x=upload_summary["PROVIDER"].unique(),y=[10] * len(upload_summary["PROVIDER"].unique()), mode='markers',marker=dict(color='red'), name='10Mbps')
combined_bar_plot_3traces(xvalues=upload_summary["PROVIDER"],
                         yvalues1=upload_summary["max"],
                         yvalues2=upload_summary["mean"],
                         yvalues3=upload_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Upload speed by provider over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Mbps",
                         xtitle="Provider",
                         line=upload_line,
                         stack=False,
                         margin=True)

In [49]:
download_summary=mean_max_median_by1(download_df,'DOWNLOAD',index_col='PROVIDER')
download_line=go.Scatter(x=upload_summary["PROVIDER"].unique(),y=[50] * len(upload_summary["PROVIDER"].unique()), mode='markers',marker=dict(color='red'), name='50Mbps')
combined_bar_plot_3traces(xvalues=download_summary["PROVIDER"],
                         yvalues1=download_summary["max"],
                         yvalues2=download_summary["mean"],
                         yvalues3=download_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Download speed by provider over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Mbps",
                         xtitle="Provider",
                         line=download_line,
                         stack=False,
                         margin=True)

In [44]:
ping_summary=mean_max_median_by1(ping_df,'PING',index_col='PROVIDER')
combined_bar_plot_3traces(xvalues=ping_summary["PROVIDER"],
                         yvalues1=ping_summary["max"],
                         yvalues2=ping_summary["mean"],
                         yvalues3=ping_summary["median"],
                         name1="Max",
                         name2="Mean",
                         name3="Median",
                         title="Ping latency by provider over the last "+time_interval+ " starting from "+ starting_point,
                         ytitle="Miliseconds",
                         xtitle="Provider",
                         stack=False,
                         margin=True)

Bell MTS has largest maximum spikes and TerraGo has largest median/average. 

### How often data is collected?

Let's ceck device number 5 with the larges number of datapoints and (8 test servers, 2 providers)  and device number 8 with largest number of providers (3) and 4 test servers?

In [45]:
upload_df.loc[upload_df['SK_PI']==5][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].head(10)

,time,SK_PI,PROVIDER,server,UPLOAD
3,2019-01-07 14:06:09-06:00,5,Bell MTS,Voyageur Internet (Winnipeg MB),0.48
14,2019-01-07 17:48:07-06:00,5,Bell MTS,Voyageur Internet (Winnipeg MB),0.53
30,2019-01-07 21:30:16-06:00,5,Bell MTS,Rogers (Winnipeg MB),0.53
33,2019-01-08 01:12:04-06:00,5,Bell MTS,Shaw Communications (Winnipeg MB),0.58
52,2019-01-08 04:54:13-06:00,5,Bell MTS,Rogers (Winnipeg MB),0.45
62,2019-01-08 08:36:11-06:00,5,Bell MTS,Rogers (Winnipeg MB),0.53
74,2019-01-08 12:18:12-06:00,5,Bell MTS,Rogers (Winnipeg MB),0.46
86,2019-01-08 16:00:14-06:00,5,Bell MTS,Voyageur Internet (Winnipeg MB),0.40
99,2019-01-08 19:42:14-06:00,5,Bell MTS,Shaw Communications (Winnipeg MB),0.59
109,2019-01-08 23:24:11-06:00,5,Bell MTS,Rogers (Winnipeg MB),0.57


In [46]:
upload_df.loc[upload_df['SK_PI']==5][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].tail(5)

,time,SK_PI,PROVIDER,server,UPLOAD
1185,2019-01-22 20:24:10-06:00,5,Bell MTS,Rogers (Winnipeg MB),0.61
1204,2019-01-23 00:06:18-06:00,5,Bell MTS,Voyageur Internet (Winnipeg MB),0.59
1207,2019-01-23 03:48:03-06:00,5,Bell MTS,Rogers (Winnipeg MB),0.41
1220,2019-01-23 07:30:08-06:00,5,Bell MTS,Rogers (Winnipeg MB),0.50
1230,2019-01-23 11:12:05-06:00,5,Bell MTS,Rogers (Winnipeg MB),0.62


In [47]:
upload_df.loc[upload_df['SK_PI']==8][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].head(5)

,time,SK_PI,PROVIDER,server,UPLOAD
230,2019-01-10 14:27:41-06:00,8,Commstream Communications,Voyageur Internet (Winnipeg MB),84.39
247,2019-01-10 19:48:11-06:00,8,MERLIN,VOI Network Solutions (Winnipeg MB),89.90
255,2019-01-10 23:30:05-06:00,8,Commstream Communications,Voyageur Internet (Winnipeg MB),120.97
272,2019-01-11 03:12:08-06:00,8,Commstream Communications,Voyageur Internet (Winnipeg MB),89.75
289,2019-01-11 06:54:12-06:00,8,MERLIN,VOI Network Solutions (Winnipeg MB),90.05


In [48]:
upload_df.loc[upload_df['SK_PI']==5][['time','SK_PI', 'PROVIDER', 'server','UPLOAD']].tail(5)

,time,SK_PI,PROVIDER,server,UPLOAD
1185,2019-01-22 20:24:10-06:00,5,Bell MTS,Rogers (Winnipeg MB),0.61
1204,2019-01-23 00:06:18-06:00,5,Bell MTS,Voyageur Internet (Winnipeg MB),0.59
1207,2019-01-23 03:48:03-06:00,5,Bell MTS,Rogers (Winnipeg MB),0.41
1220,2019-01-23 07:30:08-06:00,5,Bell MTS,Rogers (Winnipeg MB),0.50
1230,2019-01-23 11:12:05-06:00,5,Bell MTS,Rogers (Winnipeg MB),0.62


 Looks like data is collected with 3 hours 42 mins intervals using different test servers. 
 How is it determined which test server to use?